In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 读取数据集：no-filter数据、ground-truth数据集、运动控制U、测量集合Z
X_no_filter = np.load('./dataset/no_filter.npy')
X_ground_truth = np.load('./dataset/ground_truth.npy')
U = np.load('./dataset/u.npy')
Z = np.load('./dataset/z.npy')

# 无迹卡尔曼滤波

## 1. 无迹变换

无迹卡尔曼滤波通过无迹变换（Unscented Transform）来线性化。它不是通过泰勒级数展开去近似函数$g$，而是由UKF明确地从高斯中提取所谓的$\sigma$点，并将他们经过函数$g$进行变换。

通常情况下，这些$\sigma$点位于均值处及对称分布于主轴的协方差处（每维两个）。对于具有均值$\mu$和$\Sigma$的$n$维高斯分布，结果$2n+1$个$\sigma$点$\chi^{[i]}$根据如下规律进行选择：
$$\begin{align}
\chi^{[0]} &= \mu \\
\chi^{[i]} &= \mu + (\sqrt{(n + \lambda ) \Sigma})_i & i=1, \cdots , n \\
\chi^{[i]} &= \mu - (\sqrt{(n + \lambda ) \Sigma})_{i-n} & i=n+1, \cdots , 2n
\end{align}$$

式中，$\lambda = \alpha^2 (n + \kappa) - n$，$\alpha$和$\kappa$为确定$\sigma$点分布在均值多远范围内的比例参数。

每个$\sigma$点$\chi^{[i]}$有两个与之相关的权值。一个权值$\omega_m^{[i]}$，计算均值的时候使用。另一个权值$\omega_c^{[i]}$，计算高斯的协方差时使用。
$$\begin{align}
\omega_m^{[0]} &= \frac {\lambda}{n+ \lambda } \\
\omega_c^{[0]} &= \frac {\lambda}{n+ \lambda } + ( 1 - \alpha^2 + \beta ) \\
\omega_m^{[i]} &= \omega_c^{[i]} = \frac {1}{2(n+ \lambda )} \quad where \quad i = 1, \cdots , 2n \\
\end{align}$$

选择参数$\beta$对高斯表示的附加的（较高阶）分布信息进行编码。如果分布是精确的高斯分布，则$\beta = 2$是最佳选择。

$\sigma$点经过函数$g$变换，来探测$g$如何改变了高斯分布的形状。
$$y^{[i]} = g(\chi^{[i]})$$

结果高斯分布的参数$(\mu' , \Sigma')$由映射的$\sigma$点$y^{[i]}$获得：
$$\begin{align}
\mu' &= \sum_{i=0}^{2n} \omega_m^{[i]} y^{[i]} \\
\Sigma' &= \sum_{i=0}^{2n} \omega_c^{[i]} (y^{[i]} - \mu')(y^{[i]} - \mu')^T
\end{align}$$

## 2. 无迹卡尔曼滤波算法

----
01: **Algorithm Unscented_Kalman_filter**$( \mu_{t-1}, \Sigma_{t-1}, u_t, z_t )$**:**  

02: &emsp;&emsp;$\chi_{t-1} = ( \mu_{t-1} \quad \mu_{t-1} + \gamma \sqrt{\Sigma_{t-1}} \quad \mu_{t-1} - \gamma \sqrt{\Sigma_{t-1}})$  

03: &emsp;&emsp;$\bar{\chi}_t^* = g(\mu_t, \chi_{t-1})$  

04: &emsp;&emsp;$\bar{\mu}_t = \sum_{i=0}^{2n} \omega_m^{[i]} \bar{\chi}_t^{*[i]}$  

05: &emsp;&emsp;$\bar{\Sigma}_t = \sum_{i=0}^{2n} \omega_c^{[i]} (\bar{\chi}_t^{*[i]} - \bar{\mu}_t)(\bar{\chi}_t^{*[i]} - \bar{\mu}_t)^T + R_t$  

06: &emsp;&emsp;$\bar{\chi}_t = ( \bar{\mu}_t \quad \bar{\mu}_t + \gamma \sqrt{\bar{\Sigma}_t} \quad \bar{\mu}_t - \gamma \sqrt{\bar{\Sigma}_t})$  

07: &emsp;&emsp;$\bar Z_t = h(\bar{\chi}_t)$  

08: &emsp;&emsp;$\widehat{z}_t = \sum_{i=0}^{2n} \omega_m^{[i]} \bar Z_t^{[i]}$  

09: &emsp;&emsp;$S_t = \sum_{i=0}^{2n} \omega_c^{[i]} (\bar Z_t^{[i]} - \widehat z_t) (\bar Z_t^{[i]} - \widehat z_t)^T + Q_t$  

10: &emsp;&emsp;$\bar{\Sigma}_t^{x,z} = \sum_{i=0}^{2n} \omega_c^{[i]} (\bar {\chi}_t^{[i]} - \bar {\mu}_t)(\bar {Z}_t^{[i]} - \widehat {z}_t)^T$  

11: &emsp;&emsp;$K_t = \bar {\Sigma}_t^{x,z} S_t^{-1}$  

12: &emsp;&emsp;$\mu_t = \bar {\mu}_t + K_t (z_t - \widehat z_t)$  

13: &emsp;&emsp;$\Sigma_t = \bar {\Sigma}_t - K_t S_t K_t^T$  

14: &emsp;&emsp;**return**$\quad \mu_t , \Sigma_t$  

----

In [3]:

m = np.array([2, 1, .5, 1,3,0.7,.5,0.7,5]).reshape((3, 3))
e, v = np.linalg.eigh(m)
s = np.dot(v, np.diag(np.sqrt(e)))
np.dot(s, s.T)

array([[ 2. ,  1. ,  0.5],
       [ 1. ,  3. ,  0.7],
       [ 0.5,  0.7,  5. ]])